# Description

It standardize (z-score) eMERGE S-MultiXcan results projected into the MultiPLIER latent space.

# Environment variables

In [1]:
from IPython.display import display

import conf

N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

4

In [2]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=4
env: OPEN_BLAS_NUM_THREADS=4
env: NUMEXPR_NUM_THREADS=4
env: OMP_NUM_THREADS=4


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

# Settings

## Input data

## Output folder

In [5]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["DATA_TRANSFORMATIONS_DIR"], "z_score_std").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/z_score_std')

# Load eMERGE data

## Projection of S-MultiXcan z-scores

In [19]:
input_file = Path(
    conf.RESULTS["PROJECTIONS_DIR"],
    "projection-emerge-smultixcan-mashr-zscores.pkl",
).resolve()
display(input_file)
assert input_file.exists()

input_filepath_stem = input_file.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/projections/projection-emerge-smultixcan-mashr-zscores.pkl')

'projection-emerge-smultixcan-mashr-zscores'

In [20]:
pmbb_data = pd.read_pickle(input_file)

In [21]:
pmbb_data.shape

(987, 309)

In [22]:
pmbb_data.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
LV1,0.054491,-0.020097,0.022097,0.011815,-0.027529,-0.044248,0.022966,-0.052994,0.049973,0.000230,...,-0.032380,0.012214,0.006572,-0.024007,-0.006435,0.056310,0.072363,0.039763,-0.022529,0.034410
LV2,-0.045586,-0.009249,0.037598,-0.017011,-0.033393,-0.018336,0.007075,-0.011952,-0.002291,0.029972,...,0.018297,0.037166,-0.011099,-0.012876,-0.055898,0.002411,0.021665,0.011047,0.003468,0.035322
LV3,0.024822,-0.060301,-0.029168,-0.034559,0.006261,0.022122,-0.006224,0.000662,0.036817,-0.000077,...,0.038560,-0.038163,0.003248,-0.041152,0.014292,-0.039490,0.009368,0.013476,0.044817,0.004777
LV4,-0.039762,0.012785,-0.003700,0.028548,-0.029743,-0.003730,0.035600,-0.022699,0.005748,0.037000,...,-0.029203,0.075040,0.042023,-0.027431,0.004113,0.028081,0.008579,0.017368,-0.073883,-0.020724
LV5,-0.012542,-0.003956,-0.026013,0.012478,0.007327,0.024376,-0.000232,0.063066,0.044968,-0.002194,...,0.030944,-0.035498,-0.013057,-0.006033,-0.035562,-0.009096,-0.006855,-0.036068,0.002163,0.006373


# z-score standardization

In [23]:
data = pmbb_data.T

In [24]:
data_stats = data.iloc[:, :10].describe()
display(data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02
mean,1.257534e-18,5.748728e-18,-4.131898e-18,6.916438e-18,2.559980e-18,-4.491193e-19,6.287671e-18,2.874364e-18,-1.392270e-18,4.670841e-18
std,3.003813e-02,2.772754e-02,2.732986e-02,2.901664e-02,2.987249e-02,2.819247e-02,3.052889e-02,2.689039e-02,2.561742e-02,3.038666e-02
min,-8.840341e-02,-7.784959e-02,-7.319803e-02,-7.790815e-02,-7.406592e-02,-6.579391e-02,-9.306557e-02,-5.959812e-02,-6.227625e-02,-9.811341e-02
25%,-2.009651e-02,-1.725108e-02,-1.661498e-02,-2.050090e-02,-2.057211e-02,-1.709499e-02,-1.863882e-02,-1.849178e-02,-1.736913e-02,-2.025966e-02
50%,-8.071688e-04,1.684139e-03,1.123537e-03,-1.685349e-04,-2.194497e-03,1.454591e-04,-1.634808e-03,-2.537379e-04,2.284704e-04,1.140763e-03
75%,2.028215e-02,1.706551e-02,1.770317e-02,1.768648e-02,1.670923e-02,2.064828e-02,1.848979e-02,1.874173e-02,1.741150e-02,1.677429e-02
max,8.298141e-02,7.506923e-02,8.396437e-02,7.752567e-02,1.013489e-01,8.675630e-02,8.158373e-02,9.465645e-02,9.108183e-02,1.247821e-01


In [25]:
scaled_data = pd.DataFrame(
    data=scale(data), index=data.index.copy(), columns=data.columns.copy()
)

In [26]:
display(scaled_data.shape)

(309, 987)

In [27]:
display(scaled_data.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
292.3,1.816988,-1.646752,0.909722,-1.372557,-0.420517,0.967264,-0.214345,3.525794,0.424954,1.303278,...,-0.169303,1.105963,0.624641,-0.498163,-0.471079,-2.176723,0.099151,-0.409419,-0.738851,-1.034009
079,-0.670119,-0.334090,-2.209989,0.441335,-0.132648,-0.564409,-0.442529,0.290681,0.442016,-1.673754,...,0.245237,2.406511,-0.347709,-0.374762,0.918071,-0.162533,0.737962,-0.843397,-0.419225,-0.872887
741,0.736815,1.358195,-1.068975,-0.127715,-0.872216,-2.209448,-0.444954,-0.951899,0.934908,0.153968,...,1.613549,1.186504,0.137920,-0.929207,-0.935787,0.652984,-0.616765,-1.587159,0.011447,-1.446751
418.1,0.393961,-0.614508,-1.266554,0.985434,0.418391,-0.116737,-0.318609,1.531675,-0.419890,0.479685,...,-0.033526,0.242118,0.361197,-0.872287,0.504552,0.896717,-0.335216,-0.037683,-0.459028,-0.164652
280,-0.917954,-1.206278,0.229449,-1.026703,0.245660,0.118361,-1.887569,-1.181531,-0.804771,-0.181034,...,-0.527720,-1.344066,0.021140,-0.026648,0.354607,0.122197,-0.003083,-0.922225,1.436063,-0.522746


In [28]:
scaled_data_stats = scaled_data.iloc[:, :10].describe()
display(scaled_data_stats)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10
count,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02,3.090000e+02
mean,2.299491e-17,-4.886418e-17,2.012055e-17,-2.012055e-17,5.748728e-17,-1.437182e-17,-2.874364e-18,-2.874364e-18,-1.868336e-17,5.748728e-18
std,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00,1.001622e+00
min,-2.947813e+00,-2.812217e+00,-2.682661e+00,-2.689303e+00,-2.483424e+00,-2.337526e+00,-3.053387e+00,-2.219930e+00,-2.434955e+00,-3.234069e+00
25%,-6.701185e-01,-6.231734e-01,-6.089284e-01,-7.076684e-01,-6.897812e-01,-6.073509e-01,-6.115207e-01,-6.887878e-01,-6.791201e-01,-6.678102e-01
50%,-2.691506e-02,6.083738e-02,4.117690e-02,-5.817637e-03,-7.358131e-02,5.167871e-03,-5.363641e-02,-9.451310e-03,8.933025e-03,3.760246e-02
75%,6.763088e-01,6.164699e-01,6.488097e-01,6.105177e-01,5.602589e-01,7.335921e-01,6.066312e-01,6.980981e-01,6.807766e-01,5.529233e-01
max,2.767016e+00,2.711780e+00,3.077241e+00,2.676100e+00,3.398220e+00,3.082278e+00,2.676680e+00,3.525794e+00,3.561232e+00,4.113137e+00


## Testing

In [29]:
assert np.all(
    [
        np.isclose(scaled_data_stats.loc["mean", c], 0.0)
        for c in scaled_data_stats.columns
    ]
)

In [30]:
assert np.all(
    [
        np.isclose(scaled_data_stats.loc["std", c], 1.0, atol=1e-02)
        for c in scaled_data_stats.columns
    ]
)

# Save

In [31]:
output_file = Path(
    RESULTS_DIR,
    f"z_score_std-{input_filepath_stem}.pkl",
).resolve()

display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/z_score_std/z_score_std-projection-emerge-smultixcan-mashr-zscores.pkl')

In [32]:
scaled_data.to_pickle(output_file)